# Python 101
## XIII. Advanced pandas
---

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
BASE_URI = "./data/"

---
## 1. Read demographic data

- Read data from a csv file

In [ ]:
df = pd.read_csv(BASE_URI + 'population.csv')

- Select the columns we need

In [ ]:
columns = ['Data Source'] + [str(year) for year in range(1990, 2011)]
pop = df[columns].dropna()
pop.head()

- Rename `'Data Source'` to `'country'` and set it as index.

In [ ]:
pop = pop.rename(columns={'Data Source': 'country'}).set_index('country')
pop.head()

- Select a few countries

In [ ]:
countries = [ u'United Kingdom', u'Hungary', u'France', u'Germany']
subpop = pop.loc[pop.index.isin(countries)]
subpop

- Transpose the dataframe

In [ ]:
subpop = subpop.transpose()
subpop.head()

- Plot!

In [ ]:
subpop.plot();

---
## 2. Read data about alcohol

- Read the data

In [ ]:
df = pd.read_csv(BASE_URI + 'alcohol.csv')
df.head()

- Select the columns, and rename them

In [ ]:
columns = {
    'Country': 'country',
    'Year': 'year',
    'Beverage Types': 'type',
    'Display Value': 'alcohol'
}
alc = df[list(columns.keys())].rename(columns=columns).dropna()
alc.head()

- Select the same country subset

In [ ]:
subalc = alc.loc[alc['country'].isin(countries)]
subalc.head()

We only care about the combined consumption:
- filter the dataframe, select rows where `type` is `'All'`
- remove the now defunct column `type`

In [ ]:
subalc = subalc.loc[subalc['type'] == 'All']
del subalc['type']
subalc.head()

- Pivot the dataframe

In [ ]:
subalc = subalc.pivot(index='year', columns='country', values='alcohol')
subalc.head()

- Plot!

In [ ]:
subalc.plot();

---
## 3. Merge data

- Check index types

In [ ]:
print('subpop index type:', subpop.index.dtype)
print('subalc index type:', subalc.index.dtype)

subpop's index type is unicode, change it to integer

In [ ]:
subpop.index = subpop.index.astype(np.int)

- Check index lengths

In [ ]:
set(subpop.index.values).symmetric_difference(set(subalc.index.values))

Remove missing index values

In [ ]:
subpop = subpop.loc[subpop.index < 2011]
subalc = subalc.loc[subalc.index < 2011]

- Join the two dataframe!

In [ ]:
merged = subpop.join(subalc, rsuffix='_alc')
merged.head()

- Plot data into separate coordinate systems

In [ ]:
merged[['Hungary', 'Hungary_alc']].plot(subplots=True);

- Compute the total actual alcohol consumption

In [ ]:
for country in countries:
    merged[country+'_consumption'] = merged[country] * merged[country+'_alc']

- and plot it!

In [ ]:
merged[[c + '_consumption' for c in countries]].plot()

---

## 4. <a href="https://github.com/JosPolfliet/pandas-profiling">pandas-profiling</a>

#### Generate detailed reports from pandas dataframes
- import it
- generate report

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(alc)

---
## Exercises

#### 1.  Plot the top 5 alcohol consuming country in 1990 and their consumption

#### 2. Compare the average alcohol consumption in France, Germany, UK and Hungary by plotting